In [11]:
from pathlib import Path
import kagglehub

# Download latest version
path = kagglehub.dataset_download("changheonkim/iam-trocr")
path = Path(path)/"IAM"
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'iam-trocr' dataset.
Path to dataset files: /kaggle/input/iam-trocr/IAM


In [12]:
import os

# Assuming 'path' variable holds the base directory from kagglehub.dataset_download
# If not, please replace 'path' with the correct directory string, e.g., '/content/IAM'
if 'path' in globals():
    print(f"Listing directories in: {path}")
    # Use a shell command to list only directories recursively, and sort them
    !ls {path/"image"}


Listing directories in: /kaggle/input/iam-trocr/IAM
c04-110-00.jpg	e06-070-02.jpg	 g07-000b-00.jpg  n02-157-05.jpg
c04-110-01.jpg	e06-070-03.jpg	 g07-000b-01.jpg  n02-157-06.jpg
c04-110-02.jpg	e06-070-04.jpg	 g07-000b-02.jpg  n02-157-07.jpg
c04-110-03.jpg	e06-070-05.jpg	 g07-000b-03.jpg  n02-157-08.jpg
c04-116-00.jpg	e06-070-06.jpg	 g07-000b-04.jpg  n03-038-00.jpg
c04-116-01.jpg	e06-070-07.jpg	 g07-000b-05.jpg  n03-038-01.jpg
c04-116-02.jpg	e06-070-08.jpg	 g07-000b-06.jpg  n03-038-02.jpg
c04-116-03.jpg	e06-070-09.jpg	 g07-000b-07.jpg  n03-038-03.jpg
c04-134-00.jpg	f04-032-00.jpg	 g07-000b-08.jpg  n03-038-04.jpg
c04-134-01.jpg	f04-032-01.jpg	 g07-000b-09.jpg  n03-038-05.jpg
c04-134-02.jpg	f04-032-02.jpg	 g07-079a-00.jpg  n03-038-06.jpg
c04-134-03.jpg	f04-032-03.jpg	 g07-079a-01.jpg  n03-064-00.jpg
c04-134-04.jpg	f04-032-04.jpg	 g07-079a-02.jpg  n03-064-01.jpg
c04-134-05.jpg	f04-032-05.jpg	 g07-079a-03.jpg  n03-064-02.jpg
c04-134-06.jpg	f04-032-06.jpg	 g07-079a-04.jpg  n03-064-03.jpg
c04

In [13]:
import glob

# Assuming 'path' is defined and points to the base directory of the dataset
# The images are located in the 'image' subdirectory relative to 'path'
image_directory = path / "image"

# Use glob to find all .jpg files in the image directory
image_paths = sorted(glob.glob(str(image_directory / "*.jpg")))

print(f"Found {len(image_paths)} images in the dataset.")
print("First 5 image paths:")
for i, img_path in enumerate(image_paths[:5]):
    print(f"  {i+1}: {img_path}")

Found 2915 images in the dataset.
First 5 image paths:
  1: /kaggle/input/iam-trocr/IAM/image/c04-110-00.jpg
  2: /kaggle/input/iam-trocr/IAM/image/c04-110-01.jpg
  3: /kaggle/input/iam-trocr/IAM/image/c04-110-02.jpg
  4: /kaggle/input/iam-trocr/IAM/image/c04-110-03.jpg
  5: /kaggle/input/iam-trocr/IAM/image/c04-116-00.jpg


In [14]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class IAMImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB') # Load as RGB (or 'L' for grayscale if preferred)

        if self.transform:
            image = self.transform(image)

        # For now, we'll just return the image.
        # In a real scenario, you'd also load and return the corresponding label/text.
        return image

# Define transformations (you can customize these)
# Example: Resize to 224x224 and convert to tensor, then normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
])

# Instantiate your custom dataset
# 'image_paths' is assumed to be defined from a previous cell
iam_dataset = IAMImageDataset(image_paths=image_paths, transform=transform)

print(f"Number of samples in the dataset: {len(iam_dataset)}")

# To get a single image from the dataset:
single_image_tensor = iam_dataset[0]
print(f"Shape of a single image tensor: {single_image_tensor.shape}")

Number of samples in the dataset: 2915
Shape of a single image tensor: torch.Size([3, 224, 224])


In [10]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
import torch

# Load the model
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-handwritten")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/360 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

The image processor of type `DeiTImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): DeiTPatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0-11): 12 x DeiTLayer(
          (attention): DeiTAttention(
            (attention): DeiTSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense): Linear(in_features=384, out_features=1536, bias=True)
        

In [20]:
print(len(model.encoder.encoder.layer))
print(len(model.decoder.model.decoder.layers))

12
6


In [21]:
for name, module in model.named_modules():
    print(name, "->", type(module))

 -> <class 'transformers.models.vision_encoder_decoder.modeling_vision_encoder_decoder.VisionEncoderDecoderModel'>
encoder -> <class 'transformers.models.deit.modeling_deit.DeiTModel'>
encoder.embeddings -> <class 'transformers.models.deit.modeling_deit.DeiTEmbeddings'>
encoder.embeddings.patch_embeddings -> <class 'transformers.models.deit.modeling_deit.DeiTPatchEmbeddings'>
encoder.embeddings.patch_embeddings.projection -> <class 'torch.nn.modules.conv.Conv2d'>
encoder.embeddings.dropout -> <class 'torch.nn.modules.dropout.Dropout'>
encoder.encoder -> <class 'transformers.models.deit.modeling_deit.DeiTEncoder'>
encoder.encoder.layer -> <class 'torch.nn.modules.container.ModuleList'>
encoder.encoder.layer.0 -> <class 'transformers.models.deit.modeling_deit.DeiTLayer'>
encoder.encoder.layer.0.attention -> <class 'transformers.models.deit.modeling_deit.DeiTAttention'>
encoder.encoder.layer.0.attention.attention -> <class 'transformers.models.deit.modeling_deit.DeiTSelfAttention'>
encode

In [22]:
for name, param in model.named_parameters():
    print(name, param.shape)

encoder.embeddings.cls_token torch.Size([1, 1, 384])
encoder.embeddings.distillation_token torch.Size([1, 1, 384])
encoder.embeddings.position_embeddings torch.Size([1, 578, 384])
encoder.embeddings.patch_embeddings.projection.weight torch.Size([384, 3, 16, 16])
encoder.embeddings.patch_embeddings.projection.bias torch.Size([384])
encoder.encoder.layer.0.attention.attention.query.weight torch.Size([384, 384])
encoder.encoder.layer.0.attention.attention.query.bias torch.Size([384])
encoder.encoder.layer.0.attention.attention.key.weight torch.Size([384, 384])
encoder.encoder.layer.0.attention.attention.key.bias torch.Size([384])
encoder.encoder.layer.0.attention.attention.value.weight torch.Size([384, 384])
encoder.encoder.layer.0.attention.attention.value.bias torch.Size([384])
encoder.encoder.layer.0.attention.output.dense.weight torch.Size([384, 384])
encoder.encoder.layer.0.attention.output.dense.bias torch.Size([384])
encoder.encoder.layer.0.intermediate.dense.weight torch.Size([153